In [8]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')
import os

os.environ["JAVA_HOME"] = 'C:/Program Files/NetLogo 6.3.0/runtime/bin/server/'

import pyNetLogo
import ipyparallel as ipp

In [9]:
series_reporters = [
    "recovered-hospital",
    "recovered-with-help",
    "recovered-unchecked",
    "fraction-called-in",
    "deaths",
    "number-destroyed-streets-spotted",
    "fraction-destroyed-streets-spotted",
]

single_reporters = [
    'count crossings with [building-status = "collapsed"]',
    'count crossings with [building-status = "high-damage"]',
    "number-destroyed-streets",
]

In [10]:
default_values = {
    "earthquake-magnitude": 0.4,
    "amount-ambulances": 40,
    "amount-drones": 10,
}

In [11]:
replications = 1
ticks = 5

In [16]:
def run_simulation(variable, value):
    netlogo = pyNetLogo.NetLogoLink(gui=False)
    netlogo.load_model("C:/Users/Ewout/Documents/GitHub/SEN1211-earthquake-project/models_netlogo/earthquake_model.nlogo")

    netlogo.command("setup")
    netlogo.command(f"set {variable} {value}")
    single_run_data = netlogo.report(single_reporters)
    series_run_data = netlogo.repeat_report(series_reporters, ticks)
    netlogo.kill_workspace()
    return (single_run_data, series_run_data)

In [17]:
run_simulation("earthquake-magnitude", 0.4)

([2149.0, 4344.0, 2415.0],
     recovered-hospital recovered-with-help recovered-unchecked  \
 0.0                0.0                 0.0                 0.0   
 1.0                0.0                 7.0               159.0   
 2.0                0.0                12.0               307.0   
 3.0                0.0                16.0               451.0   
 4.0                0.0                22.0               608.0   
 5.0                0.0                29.0               750.0   
 
     fraction-called-in deaths number-destroyed-streets-spotted  \
 0.0                0.0   72.0                              0.0   
 1.0           0.036372   78.0                             85.0   
 2.0           0.045704   85.0                            154.0   
 3.0           0.055035   93.0                            196.0   
 4.0           0.064763   97.0                            246.0   
 5.0            0.07604  105.0                            294.0   
 
     fraction-destroyed-streets

In [13]:
single_data = {}
series_data = {}

iteration = list(range(replications))

for variable, value in default_values.items():
    for c in [0.8, 1.25]:
        value = value * c
        single_data[f"{variable}_{value:.2f}"] = []
        series_data[f"{variable}_{value:.2f}"] = []
        with ipp.Cluster(n=replications) as rc:
            print(f"Starting run: {variable} = {value:.2f}.")
            view = rc.load_balanced_view()
            data = view.map_sync(run_simulation(variable, value), iteration)
            single_data[f"{variable}_{value:.2f}"] = data[0]
            series_data[f"{variable}_{value:.2f}"] = data[1]


Starting 1 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/1 [00:00<?, ?engine/s]

Starting run: earthquake-magnitude = 0.32.
set earthquake-magnitude 0.32000000000000006
Stopping engine(s): 1674735066
engine set stopped 1674735066: {'engines': {'0': {'exit_code': 1, 'pid': 17520, 'identifier': '0'}}, 'exit_code': 1}
Stopping controller
Controller stopped: {'exit_code': 1, 'pid': 18628, 'identifier': 'ipcontroller-1674735065-4bkk-19408'}


CompositeError: one or more exceptions raised in: ([2249.0, 4288.0, 2563.0],     recovered-hospital recovered-with-help recovered-unchecked  \
0.0                0.0                 0.0                 0.0   
1.0                0.0                 8.0               141.0   
2.0                0.0                12.0               269.0   
3.0                0.0                19.0               417.0   
4.0                0.0                21.0               571.0   
5.0                0.0                29.0               707.0   

    fraction-called-in deaths number-destroyed-streets-spotted  \
0.0                0.0   90.0                              0.0   
1.0           0.035117  100.0                             80.0   
2.0           0.045117  108.0                            118.0   
3.0           0.055733  115.0                            178.0   
4.0           0.064541  125.0                            242.0   
5.0           0.074041  131.0                            287.0  

In [ ]:
series_data